## Summary generation

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


In [16]:
#get cleaned and clustered dataset of Amazon reviews
import pandas as pd

dataset_reviews = pd.read_csv("../data/top3_products.csv")  # replace with your actual path

# print(dataset_reviews[:1])
print(dataset_reviews.columns)
dataset_reviews.head()

Index(['asins', 'name', 'cluster_name', 'rating_mean', 'rating_count',
       'imageURLs', 'weighted_score', 'positive_reviews', 'negative_reviews'],
      dtype='object')


,asins,name,cluster_name,rating_mean,rating_count,imageURLs,weighted_score,positive_reviews,negative_reviews
0,B00OQVZDJM,Amazon Kindle Paperwhite - eBook reader - 4 GB...,E-Reader & Office Tablets,4.77,3175,NaN,4.732904,daughter 's 1st kindle several kindle version ...,transfer library book via usb purchased ereade...
1,B00IOY8XWQ,"Kindle Voyage E-reader, 6 High-Resolution Disp...",E-Reader & Office Tablets,4.73,599,https://i5.walmartimages.com/asr/5bb13520-39a6...,4.624555,amazing portable lightweight third amazon kind...,amazon voyage dead trying update software kind...
2,B00IOYAM4I,"Kindle Voyage E-reader, 6 High-Resolution Disp...",E-Reader & Office Tablets,4.86,51,http://pisces.bbystatic.com/image2/BestBuy_US/...,4.532403,expect improvement pricey yes ... serious read...,NaN
3,"B018SZT3BK,B01AHB9CN2","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Entertainment Tablets,4.58,2368,https://www.barcodable.com/images/barcode/0841...,4.565829,7 year old granddaughter love request 7 yr old...,color difference huge bad customer service got...
4,B01J94YIT6,"All-New Fire HD 8 Tablet with Alexa, 8 HD Disp...",Entertainment Tablets,4.58,831,https://www.barcodable.com/images/barcode/0841...,4.549514,a+ using couple 'cheaper generic android table...,stopped charging 6 month amazon fire hd 8 tabl...


In [12]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)


def get_completion(prompt, model="gpt-3.5-turbo"): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content


In [ ]:
dataset_reviews["prompt"] = dataset_reviews["positive_reviews"].fillna("") + "\n" + dataset_reviews["negative_reviews"].fillna("")
# dataset_reviews["target"] = "Summarize the product based on the reviews."

In [20]:
for category, group in dataset_reviews.groupby("name"):
    combined_reviews = " ".join(group["prompt"].astype(str).tolist())

    prompt_1 = f"""
    Your task is to generate a short summary of the three products from a single category.
    It should read like a review.
    Write it in the style of product reviews on Consumer Reviews website, The Verge, The Wirecutter.

    What are the best qualities of each of those products?
    You should also identify top complaints for each of those products.
    What is the worst product in the category and why you should never buy it.

    Summarize the review below, delimited by triple backticks, in at most 200 words.

    Review: ```{combined_reviews}```
    """

    response = get_completion(prompt_1)
    print(f"\n=== Summary for Category: {category} ===\n")
    print(response)
prompt_2 = f"""
Your task is to generate a short summary of the three product from a single category \
It should read like a review.
Write it in the style of product reviews on Consumer Reviews website, The Verge, The Wirecutter

What are the best qualities of each of those products.
You should also identify top complaints for each of those products.
What is the worst product in the category and why you should never buy it

Summarize the review below, delimited by triple 
backticks, in at most 200 words. 

Review: ```{dataset_reviews["prompt"]}```
"""

# response = get_completion(prompt_2)
# print(response)


=== Summary for Category: All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Blue Kid-Proof Case ===

The Amazon Fire HD 8 Kids Edition tablet is a must-have for parents looking for a kid-friendly device with excellent parental controls and age-appropriate content. The tablet comes with a 2-year worry-free warranty and a free 1-year subscription to Amazon Kids, offering unlimited access to children's shows and apps. It is durable and comes with a thick foam case, making it ideal for young children. However, the screen brightness could be improved, and it does not support YouTube Red video downloads.

In comparison, the Apple iPad is more expensive but offers a wider range of apps and features. It is not as durable as the Amazon Fire tablet, but it comes with a worry-free warranty and is a great value for older children. The downside is the limited app selection in the App Store.

The Nabi tablet is a good option for younger children, with easy-to-use parental controls and a s